In [1]:
import cv2
import pickle
import os.path
import numpy as np
import imutils
from imutils import paths
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten, Dense
from keras.backend.tensorflow_backend import set_session

Using TensorFlow backend.


In [2]:
def resize_to_fit(image, width, height):
    """
    A helper function to resize an image to fit within a given size
    :param image: image to resize
    :param width: desired width in pixels
    :param height: desired height in pixels
    :return: the resized image
    """

    # grab the dimensions of the image, then initialize
    # the padding values
    (h, w) = image.shape[:2]

    # if the width is greater than the height then resize along
    # the width
    if w > h:
        image = imutils.resize(image, width=width)

    # otherwise, the height is greater than the width so resize
    # along the height
    else:
        image = imutils.resize(image, height=height)
        
    

    # determine the padding values for the width and height to
    # obtain the target dimensions
    padW = int((width - image.shape[1]) / 2.0)
    padH = int((height - image.shape[0]) / 2.0)

    # pad the image then apply one more resizing to handle any
    # rounding issues
    image = cv2.copyMakeBorder(image, padH, padH, padW, padW,
        cv2.BORDER_REPLICATE)
    image = cv2.resize(image, (width, height))

    # return the pre-processed image
    return image

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

rootdir = r"D:\Python\Jupyter"

In [4]:
data = []
labels = []
pixel_size = 256

for folder in os.listdir(os.path.join("/", rootdir, "training")):
    print("Reading from:", folder)
    
    for image_file in paths.list_images(os.path.join("/", os.path.join("/", rootdir, "training"), folder)):
        # Load the image and convert it to grayscale
        
        image = cv2.imread(image_file)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Resize the letter so it fits in a pixel box
        image = resize_to_fit(image, pixel_size, pixel_size)

        # Add a third channel dimension to the image to make Keras happy
        image = np.expand_dims(image, axis=2)
        image = np.repeat(image, 3, axis=2)

        # Grab the name of the letter based on the folder it was in
        label = folder

        # Add the image and it's label to our training data
        data.append(image)
        labels.append(label)
        
print(len(data), len(labels))

Reading from: bedroom
Reading from: Coast
Reading from: Forest
Reading from: Highway
Reading from: industrial
Reading from: Insidecity
Reading from: kitchen
Reading from: livingroom
Reading from: Mountain
Reading from: Office
Reading from: OpenCountry
Reading from: store
Reading from: Street
Reading from: Suburb
Reading from: TallBuilding
1500 1500


In [5]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

(X_train, X_test, Y_train, Y_test) = train_test_split(data, labels, test_size=0.2, random_state=0)

lb = LabelBinarizer().fit(Y_train)
Y_train = lb.transform(Y_train)
Y_test = lb.transform(Y_test)

with open(os.path.join("/", rootdir, "model_labels.dat"), "wb") as f:
    pickle.dump(lb, f)

In [6]:
# Transfer learning
from keras import applications

model = applications.ResNet50(weights = "imagenet", include_top=False, input_shape = (pixel_size, pixel_size, 3))

print(model.summary())

C:\Users\vladc\Anaconda3\envs\deeplearning\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 128, 128, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [7]:
# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
from keras.layers import Dropout, Input
from keras.models import Model 
    
#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(15, activation="softmax")(x)

model = Model(input = model.input, output = predictions)

C:\Users\vladc\Anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  del sys.path[0]


In [8]:
# Checkpoint
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("vc1g16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

In [9]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=32, epochs=1, verbose=1)

Train on 1200 samples, validate on 300 samples
Epoch 1/1
1200/1200 [==============================] - 1721s 1s/step - loss: 5.1658 - acc: 0.1067 - val_loss: 13.7242 - val_acc: 0.0467


In [11]:
model.save(os.path.join("/", rootdir, "res_1.hdf5"))

In [12]:
# Load up the model labels
with open(os.path.join("/", rootdir, "model_labels.dat"), "rb") as f:
    lb = pickle.load(f)

In [13]:
scores = model.evaluate(X_test, Y_test, batch_size=32, verbose=1)
print("\nAccuracy: %.4f%%\n\n"%(scores[1]*100))

300/300 [==============================] - 95s 315ms/step

Accuracy: 4.6667%




In [14]:
# Epoch 2
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=32, epochs=1, verbose=1)

Train on 1200 samples, validate on 300 samples
Epoch 1/1
1200/1200 [==============================] - 1747s 1s/step - loss: 4.0203 - acc: 0.1133 - val_loss: 15.3122 - val_acc: 0.0500


In [15]:
model.save(os.path.join("/", rootdir, "res_2.hdf5"))

In [16]:
scores = model.evaluate(X_test, Y_test, batch_size=32, verbose=1)
print("\nAccuracy: %.4f%%\n\n"%(scores[1]*100))

300/300 [==============================] - 89s 295ms/step

Accuracy: 5.0000%


